In [1]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os

# BoundaryDiagrams

In [2]:
data_files = glob('./data/*.pkl')

In [3]:
len(data_files)

348

In [4]:
def edge_hueristic(img, filtration_num, percent_to_take):
    """
    :param img: Numpy array representing the image.
    :param filtration_num: Number used to represent non background color for the image.
    :param percent_to_take: number between 0-1 that represents percentage of edges to take.
    """
    edge_img = np.zeros(shape=img.shape)
    # Go through each pixel that is not a boundary.
    for ix in np.argwhere(img > filtration_num):
        # Grad a 3x3 grid around the pixel.
        grid = img[ix[0]-1: ix[0]+2, 
                   ix[1]-1: ix[1]+2]
        # If any background colors are near the point its on a boundary location.
        if 0 in grid and np.random.rand() <= percent_to_take:
            edge_img[ix[0], ix[1]] = 1
    return edge_img

In [7]:
def clean_data(data):
    data[data == -1] = 0
    data[143:167, 223:247][data[143:167, 223:247] == 0] = 1
    return data 

# Generating point clouds

In [45]:
for f in data_files:
    d = np.load(f)
    d = clean_data(d)
    
    edge_d = edge_hueristic(d, filtration_num=0, percent_to_take=1)
    np.savetxt('./point_clouds/' + f.split('\\')[-1] + ".txt", np.argwhere(edge_d == 1), fmt='%d')

# Processing point clouds through ripser.

In [7]:
for file in glob('./point_clouds/*'):
    already_processed = glob('./barcodes/*')
    file_name = file.split('\\')[-1].split('.')[0]
    if "./barcodes\\{}_barcode.txt".format(file_name) in already_processed:
        print("Skipping {}".format(file_name))
    else:
        print("Working on {}".format(file_name))
        os.system('ripser.exe point_clouds/{point_cloud_file}.pkl.txt --format point-cloud --dim 1 --threshold 500 >> barcodes/{dest_file}'.format(point_cloud_file=file_name, dest_file=file_name + '_barcode.txt'))

Skipping 1989_01_SIN_data
Skipping 1989_02_SIN_data
Skipping 1989_03_SIN_data
Skipping 1989_04_SIN_data
Skipping 1989_05_SIN_data
Skipping 1989_06_SIN_data
Skipping 1989_07_SIN_data
Skipping 1989_08_SIN_data
Skipping 1989_09_SIN_data
Skipping 1989_10_SIN_data
Skipping 1989_11_SIN_data
Skipping 1989_12_SIN_data
Skipping 1990_01_SIN_data
Skipping 1990_02_SIN_data
Skipping 1990_03_SIN_data
Skipping 1990_04_SIN_data
Skipping 1990_05_SIN_data
Skipping 1990_06_SIN_data
Skipping 1990_07_SIN_data
Skipping 1990_08_SIN_data
Skipping 1990_09_SIN_data
Skipping 1990_10_SIN_data
Skipping 1990_11_SIN_data
Skipping 1990_12_SIN_data
Skipping 1991_01_SIN_data
Skipping 1991_02_SIN_data
Skipping 1991_03_SIN_data
Skipping 1991_04_SIN_data
Skipping 1991_05_SIN_data
Skipping 1991_06_SIN_data
Skipping 1991_07_SIN_data
Skipping 1991_08_SIN_data
Skipping 1991_09_SIN_data
Skipping 1991_10_SIN_data
Skipping 1991_11_SIN_data
Skipping 1991_12_SIN_data
Skipping 1992_01_SIN_data
Skipping 1992_02_SIN_data
Skipping 199

# Need to split ripser outputs into dim0 and dim1 barcodes only.

### what to do about [0,) ?????

In [10]:
def ripserstr_to_numpy(s):
    "Takes in ripser output with extraneious header info removed, and converts to numpy array of size (?,2)"
    return np.fromstring(s.replace('[', '').replace(')', '').replace(',', ', ').replace('\n',', '), dtype=np.float32, sep=',').reshape((-1, 2))

In [12]:
for filename in glob('barcodes/*.txt'):
    # Reading in barecode data.
    f = open(filename, 'r')
    file = f.read()
    f.close()
    
    # Making new file name from old.
    newfile_name = filename.split('\\')[-1].replace('_barcode.txt', '')
    
    # Splitting files between dim0 and dim1. 
    split_files = file.split('persistence intervals in dim 1:')
    
    # Retrieve just the dim elements and replace any infinite positions with a large persistance (100 in our case.) for dim0.
    dim0 = split_files[0].split('dim 0:')[1].lstrip('\n').rstrip('\n').replace(' ', '').replace(',)', ',100)')
    dim0 = ripserstr_to_numpy(dim0)
    assert(dim0.shape[1] == 2)
    np.save('./barcodes/dim0/{}'.format(newfile_name + '_dim0'), dim0)
    
    # Dim1 
    dim1 = split_files[1].lstrip('\n').rstrip('\n').replace(' ', '')
    dim1 = ripserstr_to_numpy(dim1)
    assert(dim1.shape[1] == 2)
    np.save('./barcodes/dim1/{}'.format(newfile_name + '_dim1'), dim1)
    print(newfile_name, ": ", dim0.shape, ", ", dim1.shape)

1989_01_SIN_data :  (6247, 2) ,  (1026, 2)
1989_02_SIN_data :  (6248, 2) ,  (984, 2)
1989_03_SIN_data :  (6351, 2) ,  (1022, 2)
1989_04_SIN_data :  (5967, 2) ,  (946, 2)
1989_05_SIN_data :  (5054, 2) ,  (849, 2)
1989_06_SIN_data :  (4534, 2) ,  (751, 2)
1989_07_SIN_data :  (3948, 2) ,  (660, 2)
1989_08_SIN_data :  (3553, 2) ,  (669, 2)
1989_09_SIN_data :  (2778, 2) ,  (478, 2)
1989_10_SIN_data :  (3636, 2) ,  (639, 2)
1989_11_SIN_data :  (4518, 2) ,  (734, 2)
1989_12_SIN_data :  (5637, 2) ,  (952, 2)
1990_01_SIN_data :  (6319, 2) ,  (1042, 2)
1990_02_SIN_data :  (6325, 2) ,  (995, 2)
1990_03_SIN_data :  (6239, 2) ,  (934, 2)
1990_04_SIN_data :  (6031, 2) ,  (927, 2)
1990_05_SIN_data :  (5052, 2) ,  (787, 2)
1990_06_SIN_data :  (4751, 2) ,  (831, 2)
1990_07_SIN_data :  (4008, 2) ,  (695, 2)
1990_08_SIN_data :  (3593, 2) ,  (724, 2)
1990_09_SIN_data :  (2590, 2) ,  (485, 2)
1990_10_SIN_data :  (3777, 2) ,  (698, 2)
1990_11_SIN_data :  (4633, 2) ,  (793, 2)
1990_12_SIN_data :  (5781, 2) ,